In [1]:
import pandas as pd
import numpy as np
import warnings

warnings.simplefilter('ignore')

raw_prices = pd.read_csv(r'C:\Users\Andre\Documents\LowVolatility\Raw_prices.csv')
negocios = pd.read_csv(r'C:\Users\Andre\Documents\LowVolatility\Negocios.csv')
financeiro = pd.read_csv(r'C:\Users\Andre\Documents\LowVolatility\Financeiro.csv')
ibov = pd.read_csv(r'C:\Users\Andre\Documents\LowVolatility\Ibov_data.csv')

In [2]:
raw_prices['Datas'] = pd.to_datetime(raw_prices['Datas'])

In [3]:
raw_prices.head()

,Datas,RRRP3,TTEN3,QVUM3B,QVQP3B,APPA3,APPA4,ABCB3,ABCB4,ABCB11,...,WLMM4,WWOW3,ILMD3,ILMD4,YBRA3B,YDUQ3,ESTC4,ESTC11,OPZI3B,ZIVI4
0,1986-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1986-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.862124,NaN,NaN,98.087575,NaN,NaN,NaN,NaN,NaN,NaN
2,1986-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,96.125827,NaN,NaN,NaN,NaN,NaN,NaN
3,1986-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.892488,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1986-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.350428,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
raw_prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9303 entries, 0 to 9302
Columns: 1383 entries, Datas to ZIVI4
dtypes: datetime64[ns](1), float64(1382)
memory usage: 98.2 MB


In [5]:
precos_corte = raw_prices[(raw_prices['Datas'] >= '2000-01-01') & (raw_prices['Datas'] <= '2020-12-31')]

In [6]:
long_data = precos_corte.melt(id_vars=['Datas'], var_name='Codigo', value_name='Preco')

In [7]:
long_data['Ano'] = long_data['Datas'].dt.year

In [8]:
grouped_data = long_data.groupby(['Codigo', 'Ano'])

In [9]:
long_data['Total_Dias_Negociacao'] = long_data.groupby(['Codigo', 'Ano'])['Datas'].transform('nunique')
long_data['Pregoes'] = long_data.groupby(['Codigo', 'Ano'])['Preco'].transform(lambda x: x.notna().sum())
long_data['Pregoes_Percentual'] = long_data['Pregoes'] / long_data['Total_Dias_Negociacao']


In [10]:
long_data = long_data[long_data['Pregoes_Percentual'] > 0.9]

In [11]:
long_data = long_data.reset_index(drop=True)

In [12]:
long_data['Preco'] = long_data['Preco'].fillna(method='ffill')
long_data['Retorno'] = long_data.groupby('Codigo')['Preco'].pct_change()
long_data['Risco'] = long_data.groupby(['Codigo', 'Ano'])['Retorno'].transform(lambda x: np.sqrt(len(x)) * np.nanstd(x))

In [13]:
long_data['Retorno_positivos'] = long_data.groupby(['Codigo', 'Ano'])['Retorno'].transform(lambda x: (x > 0).sum())
long_data['Retorno_negativos'] = long_data.groupby(['Codigo', 'Ano'])['Retorno'].transform(lambda x: (x < 0).sum())
long_data['Perc_positivos'] = long_data['Retorno_positivos'] / (long_data['Retorno_positivos'] + long_data['Retorno_negativos'])
long_data['Perc_negativos'] = long_data['Retorno_negativos'] / (long_data['Retorno_positivos'] + long_data['Retorno_negativos'])
long_data['Retorno'] = long_data['Retorno'].fillna(0)
long_data['Retorno_Anual'] = long_data.groupby(['Codigo', 'Ano'])['Retorno'].transform(lambda x: np.prod(1 + x) - 1)
long_data['Retorno_acumulado'] = long_data.groupby('Codigo')['Retorno'].transform(lambda x: (1 + x).cumprod() - 1)
long_data['FIP'] = long_data['Perc_negativos'] - long_data['Perc_positivos']

In [14]:
dados_agregados = long_data.groupby(['Codigo', 'Ano']).agg({'Retorno_Anual':'first', 'Risco':'first', 'FIP':'first'}).reset_index()

In [15]:
dados_agregados['Decil_Retorno'] = dados_agregados.groupby('Ano')['Retorno_Anual'].transform(lambda x: pd.qcut(x, 10, labels=False, duplicates='drop') + 1)
dados_agregados['Decil_Risco'] = dados_agregados.groupby('Ano')['Risco'].transform(lambda x: pd.qcut(x, 10, labels=False, duplicates='drop') + 1)
dados_agregados['Decil_FIP'] = dados_agregados.groupby(['Codigo', 'Ano'])['FIP'].transform(lambda x: pd.qcut(x, 10, labels=False, duplicates='drop') + 1)

In [16]:
dados_agregados['Decil_Retorno'] = dados_agregados.groupby('Ano')['Retorno_Anual'].transform(lambda x: pd.qcut(x, 10, labels=False, duplicates='drop') + 1)
dados_agregados['Decil_Risco'] = dados_agregados.groupby('Ano')['Risco'].transform(lambda x: pd.qcut(x, 10, labels=False, duplicates='drop') + 1)
dados_agregados['Decil_FIP'] = dados_agregados.groupby('Ano')['FIP'].transform(lambda x: pd.qcut(x, 10, labels=False, duplicates='drop') + 1)


In [17]:
long_data = pd.merge(long_data, dados_agregados[['Codigo', 'Ano', 'Decil_Retorno', 'Decil_Risco', 'Decil_FIP']], on=['Codigo', 'Ano'], how='left')

In [18]:
long_data_2 = long_data.groupby(['Codigo', 'Ano']).first().reset_index().drop(['Preco'], axis=1)

In [19]:
long_data_2

,Codigo,Ano,Datas,Total_Dias_Negociacao,Pregoes,Pregoes_Percentual,Retorno,Risco,Retorno_positivos,Retorno_negativos,Perc_positivos,Perc_negativos,Retorno_Anual,Retorno_acumulado,FIP,Decil_Retorno,Decil_Risco,Decil_FIP
0,AALR3,2017,2017-01-02,260,246,0.946154,0.000000,0.285582,104,123,0.458150,0.541850,-0.015323,0.000000,0.083700,3,3,9
1,AALR3,2018,2018-01-01,261,245,0.938697,0.000000,0.247384,107,126,0.459227,0.540773,-0.133144,-0.015323,0.081545,4,1,7
2,AALR3,2019,2019-01-01,261,248,0.950192,0.000000,0.255844,122,111,0.523605,0.476395,0.315486,-0.146427,-0.047210,5,2,4
3,AALR3,2020,2020-01-01,262,249,0.950382,0.000000,0.668783,116,130,0.471545,0.528455,-0.410348,0.122864,0.056911,2,6,7
4,ABCB4,2008,2008-01-01,262,249,0.950382,0.000000,0.648352,112,131,0.460905,0.539095,-0.552452,0.000000,0.078189,5,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4002,YDUQ3,2016,2016-01-01,261,249,0.954023,0.000000,0.533260,127,120,0.514170,0.485830,0.202629,-0.595476,-0.028340,5,7,4
4003,YDUQ3,2017,2017-01-02,260,246,0.946154,-0.022152,0.454226,135,108,0.555556,0.444444,1.052801,-0.524285,-0.111111,10,7,2
4004,YDUQ3,2018,2018-01-01,261,245,0.938697,0.000000,0.446707,122,122,0.500000,0.500000,-0.252485,-0.001329,0.000000,3,6,4
4005,YDUQ3,2019,2019-01-01,261,248,0.950192,0.000000,0.417148,124,122,0.504065,0.495935,0.952263,-0.253479,-0.008130,9,7,6


In [20]:
negocios['Datas'] = pd.to_datetime(negocios['Datas'])
negocios = negocios[(negocios['Datas'] >= '2000-01-01') & (negocios['Datas'] <= '2020-12-31')]
negocios = negocios.melt(id_vars=['Datas'], var_name='Codigo', value_name='Negocios')
negocios['Ano'] = negocios['Datas'].dt.year
negocios['Media_Negocios_Acao'] = negocios.groupby(['Codigo', 'Ano'])['Negocios'].transform(lambda x: x.mean(skipna=True))
negocios['Dias_Negocios'] = negocios.groupby(['Codigo', 'Ano'])['Negocios'].transform(lambda x: x.notna().sum())
dias_por_ano = negocios.groupby('Ano')['Datas'].nunique().rename('Total_Dias_Negociacao')
negocios = negocios.merge(dias_por_ano, on='Ano', how='left')
negocios['Negocios_Percentual'] = negocios['Dias_Negocios'] / negocios['Total_Dias_Negociacao']


In [21]:
negocios= negocios[negocios['Negocios_Percentual'] > 0.9]

In [22]:
negocios['Media_Negocios'] = negocios.groupby('Ano')['Negocios'].transform(lambda x: x.mean(skipna=True))

In [23]:
negocios['Razao_Media_Negocios'] = negocios['Media_Negocios_Acao'] / negocios['Media_Negocios']

In [24]:
negocios

,Datas,Codigo,Negocios,Ano,Media_Negocios_Acao,Dias_Negocios,Total_Dias_Negociacao,Negocios_Percentual,Media_Negocios,Razao_Media_Negocios
40439,2008-01-01,ABCB4,NaN,2008,56.943775,249,262,0.950382,738.928937,0.077063
40440,2008-01-02,ABCB4,42.0,2008,56.943775,249,262,0.950382,738.928937,0.077063
40441,2008-01-03,ABCB4,10.0,2008,56.943775,249,262,0.950382,738.928937,0.077063
40442,2008-01-04,ABCB4,23.0,2008,56.943775,249,262,0.950382,738.928937,0.077063
40443,2008-01-07,ABCB4,18.0,2008,56.943775,249,262,0.950382,738.928937,0.077063
...,...,...,...,...,...,...,...,...,...,...
7550057,2020-12-25,YDUQ3,NaN,2020,17584.401606,249,262,0.950382,8966.006332,1.961230
7550058,2020-12-28,YDUQ3,7092.0,2020,17584.401606,249,262,0.950382,8966.006332,1.961230
7550059,2020-12-29,YDUQ3,14530.0,2020,17584.401606,249,262,0.950382,8966.006332,1.961230
7550060,2020-12-30,YDUQ3,10899.0,2020,17584.401606,249,262,0.950382,8966.006332,1.961230


In [25]:
negocios_2 = negocios.groupby(['Codigo', 'Ano']).first().reset_index().drop(['Negocios'], axis=1)

In [26]:
negocios_2

,Codigo,Ano,Datas,Media_Negocios_Acao,Dias_Negocios,Total_Dias_Negociacao,Negocios_Percentual,Media_Negocios,Razao_Media_Negocios
0,AALR3,2017,2017-01-02,690.300813,246,260,0.946154,3837.695353,0.179874
1,AALR3,2018,2018-01-01,651.787755,245,261,0.938697,4618.601414,0.141122
2,AALR3,2019,2019-01-01,699.955645,248,261,0.950192,5521.179429,0.126776
3,AALR3,2020,2020-01-01,2232.979920,249,262,0.950382,8966.006332,0.249050
4,ABCB4,2008,2008-01-01,56.943775,249,262,0.950382,738.928937,0.077063
...,...,...,...,...,...,...,...,...,...
4002,YDUQ3,2016,2016-01-01,8125.566265,249,261,0.954023,4196.871848,1.936101
4003,YDUQ3,2017,2017-01-02,10291.130081,246,260,0.946154,3837.695353,2.681591
4004,YDUQ3,2018,2018-01-01,14408.710204,245,261,0.938697,4618.601414,3.119713
4005,YDUQ3,2019,2019-01-01,12155.060484,248,261,0.950192,5521.179429,2.201533


In [27]:
financeiro['Datas'] = pd.to_datetime(financeiro['Datas'])
financeiro = financeiro[(financeiro['Datas'] >= '2000-01-01') & (financeiro['Datas'] <= '2020-12-31')]


In [28]:
financeiro = financeiro.melt(id_vars=['Datas'], var_name='Codigo', value_name='Financeiro')

In [29]:
financeiro['Ano'] = financeiro['Datas'].dt.year

In [30]:
financeiro['Media_Financeiro_Ativo_Ano'] = financeiro.groupby(['Codigo', 'Ano'])['Financeiro'].transform(lambda x: x.mean(skipna=True))

In [31]:
financeiro['Financeiro_Dias'] = financeiro.groupby(['Codigo', 'Ano'])['Financeiro'].transform(lambda x: x.notna().sum())

In [32]:
financeiro['Total_Financeiro_Dias'] = financeiro.groupby(['Codigo', 'Ano'])['Datas'].transform(lambda x: x.nunique())

In [33]:
financeiro['Financeiro_Dias_Percentual'] = financeiro['Financeiro_Dias'] / financeiro['Total_Financeiro_Dias']

In [34]:
financeiro = financeiro[financeiro['Financeiro_Dias_Percentual'] > 0.9]

In [35]:
financeiro['Media_Financeiro_Ano'] = financeiro.groupby('Ano')['Financeiro'].transform(lambda x: x.mean(skipna=True))
financeiro['DP_Financeiro_Ano'] = financeiro.groupby('Ano')['Financeiro'].transform(lambda x: x.std(skipna=True))

In [36]:
financeiro['Razao_Financeiro'] = financeiro['Media_Financeiro_Ativo_Ano'] / financeiro['Media_Financeiro_Ano']

In [37]:
financeiro_2 = financeiro.groupby(['Codigo', 'Ano']).first().reset_index().drop('Financeiro', axis=1)

In [38]:
financeiro_2

,Codigo,Ano,Datas,Media_Financeiro_Ativo_Ano,Financeiro_Dias,Total_Financeiro_Dias,Financeiro_Dias_Percentual,Media_Financeiro_Ano,DP_Financeiro_Ano,Razao_Financeiro
0,AALR3,2017,2017-01-02,4715.336297,246,260,0.946154,31612.789691,82780.268217,0.149159
1,AALR3,2018,2018-01-01,2721.427159,245,261,0.938697,44995.468341,147739.452652,0.060482
2,AALR3,2019,2019-01-01,2882.498875,248,261,0.950192,56516.776625,161699.065053,0.051003
3,AALR3,2020,2020-01-01,7994.316940,249,262,0.950382,84271.693786,228090.935231,0.094864
4,ABCB4,2008,2008-01-01,1668.837213,249,262,0.950382,20404.614073,80778.054730,0.081787
...,...,...,...,...,...,...,...,...,...,...
4002,YDUQ3,2016,2016-01-01,35680.941169,249,261,0.954023,30115.231081,81544.710207,1.184814
4003,YDUQ3,2017,2017-01-02,68721.473049,246,260,0.946154,31612.789691,82780.268217,2.173850
4004,YDUQ3,2018,2018-01-01,98637.096163,245,261,0.938697,44995.468341,147739.452652,2.192156
4005,YDUQ3,2019,2019-01-01,91065.786169,248,261,0.950192,56516.776625,161699.065053,1.611305


In [39]:
df = long_data_2.merge(negocios_2, on=['Codigo', 'Ano'], how='left')

In [40]:
df = df.merge(financeiro_2, on=['Codigo', 'Ano'], how='left')

In [41]:
df = df.drop(['Total_Dias_Negociacao_y', 'Datas_y', 'Datas'], axis=1).rename(columns={'Datas_x':'Datas', 'Total_Dias_Negociacao_x':'Total_Dias_Negociacao'})

In [42]:
df

,Codigo,Ano,Datas,Total_Dias_Negociacao,Pregoes,Pregoes_Percentual,Retorno,Risco,Retorno_positivos,Retorno_negativos,...,Negocios_Percentual,Media_Negocios,Razao_Media_Negocios,Media_Financeiro_Ativo_Ano,Financeiro_Dias,Total_Financeiro_Dias,Financeiro_Dias_Percentual,Media_Financeiro_Ano,DP_Financeiro_Ano,Razao_Financeiro
0,AALR3,2017,2017-01-02,260,246,0.946154,0.000000,0.285582,104,123,...,0.946154,3837.695353,0.179874,4715.336297,246,260,0.946154,31612.789691,82780.268217,0.149159
1,AALR3,2018,2018-01-01,261,245,0.938697,0.000000,0.247384,107,126,...,0.938697,4618.601414,0.141122,2721.427159,245,261,0.938697,44995.468341,147739.452652,0.060482
2,AALR3,2019,2019-01-01,261,248,0.950192,0.000000,0.255844,122,111,...,0.950192,5521.179429,0.126776,2882.498875,248,261,0.950192,56516.776625,161699.065053,0.051003
3,AALR3,2020,2020-01-01,262,249,0.950382,0.000000,0.668783,116,130,...,0.950382,8966.006332,0.249050,7994.316940,249,262,0.950382,84271.693786,228090.935231,0.094864
4,ABCB4,2008,2008-01-01,262,249,0.950382,0.000000,0.648352,112,131,...,0.950382,738.928937,0.077063,1668.837213,249,262,0.950382,20404.614073,80778.054730,0.081787
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4002,YDUQ3,2016,2016-01-01,261,249,0.954023,0.000000,0.533260,127,120,...,0.954023,4196.871848,1.936101,35680.941169,249,261,0.954023,30115.231081,81544.710207,1.184814
4003,YDUQ3,2017,2017-01-02,260,246,0.946154,-0.022152,0.454226,135,108,...,0.946154,3837.695353,2.681591,68721.473049,246,260,0.946154,31612.789691,82780.268217,2.173850
4004,YDUQ3,2018,2018-01-01,261,245,0.938697,0.000000,0.446707,122,122,...,0.938697,4618.601414,3.119713,98637.096163,245,261,0.938697,44995.468341,147739.452652,2.192156
4005,YDUQ3,2019,2019-01-01,261,248,0.950192,0.000000,0.417148,124,122,...,0.950192,5521.179429,2.201533,91065.786169,248,261,0.950192,56516.776625,161699.065053,1.611305


In [43]:
ibov['Datas'] = pd.to_datetime(ibov['Datas'])
ibov = ibov[(ibov['Datas'] >= '2000-01-01') & (ibov['Datas'] <= '2020-12-31')]
ibov['Pontos'] = ibov['Pontos'].fillna(method='ffill')
ibov['Retorno'] = ibov['Pontos'].pct_change()


In [44]:
ibov['Retorno'] = ibov['Retorno'].fillna(0)

In [45]:
ibov['Ret_Acumulado_Carteira'] = (1 + ibov['Retorno']).cumprod() - 1

In [46]:
ibov = ibov.reset_index(drop=True)

In [47]:
df_filtrado = df[(df['Razao_Financeiro'] > 0.05) & (df['Razao_Media_Negocios'] > 0.01)]

In [48]:
lista_dfs = [grupo.reset_index(drop=True) for _, grupo in df_filtrado.groupby('Ano')]

In [49]:
lista_dfs

[   Codigo   Ano      Datas  Total_Dias_Negociacao  Pregoes  \
 0   ACES3  2000 2000-01-03                    260      248   
 1   ACES4  2000 2000-01-03                    260      248   
 2   AMBV4  2000 2000-01-03                    260      247   
 3   ARCE4  2000 2000-01-03                    260      243   
 4   ARCZ6  2000 2000-01-03                    260      248   
 ..    ...   ...        ...                    ...      ...   
 91  VCPA4  2000 2000-01-03                    260      247   
 92  VIVO3  2000 2000-01-03                    260      248   
 93  VIVO4  2000 2000-01-03                    260      248   
 94  VIVT3  2000 2000-01-03                    260      248   
 95  VIVT4  2000 2000-01-03                    260      248   
 
     Pregoes_Percentual  Retorno     Risco  Retorno_positivos  \
 0             0.953846      0.0  0.559740                 91   
 1             0.953846      0.0  0.561996                101   
 2             0.950000      0.0  0.460710     

In [50]:
# Supondo que Lista_DFs seja sua lista de DataFrames
Lista_Carteira_Mercado = []

for df in lista_dfs:
    agrupado = df.groupby('Codigo').agg({
        'Risco': 'mean',
        'Retorno_Anual': 'mean',
        'Decil_Retorno': 'mean',
        'Decil_Risco': 'mean',
        'Ano': 'mean'
    }).reset_index()
    Lista_Carteira_Mercado.append(agrupado)


In [51]:
Lista_Carteira_Mercado

[   Codigo     Risco  Retorno_Anual  Decil_Retorno  Decil_Risco     Ano
 0   ACES3  0.559740      -0.045142            6.0          6.0  2000.0
 1   ACES4  0.561996      -0.109289            5.0          6.0  2000.0
 2   AMBV4  0.460710       0.707908           10.0          4.0  2000.0
 3   ARCE4  0.540558       0.180894            8.0          6.0  2000.0
 4   ARCZ6  0.420268      -0.418092            2.0          2.0  2000.0
 ..    ...       ...            ...            ...          ...     ...
 91  VCPA4  0.497431      -0.342758            2.0          5.0  2000.0
 92  VIVO3  0.604016      -0.282104            3.0          7.0  2000.0
 93  VIVO4  0.656661      -0.340560            2.0          8.0  2000.0
 94  VIVT3  0.471792      -0.263718            3.0          4.0  2000.0
 95  VIVT4  0.508539      -0.366347            2.0          5.0  2000.0
 
 [96 rows x 6 columns],
    Codigo     Risco  Retorno_Anual  Decil_Retorno  Decil_Risco     Ano
 0   ABEV3  1.023116      -0.942209   

In [52]:
lista_carteira_1 = []
lista_carteira_2 = []
lista_carteira_3 = []
lista_carteira_4 = []
lista_carteira_5 = []

for df in lista_dfs:
    # Para a Carteira 1
    filtrado_1 = df[df['Decil_Risco'] <= 1]
    agregado_1 = filtrado_1.groupby('Codigo').agg({
        'Risco': 'mean',
        'Retorno_Anual': 'mean',
        'Decil_Retorno': 'mean',
        'Decil_Risco': 'mean',
        'Ano': 'mean'
    }).reset_index()
    lista_carteira_1.append(agregado_1)

    # Para a Carteira 2
    filtrado_2 = df[(df['Decil_Risco'] <= 2) & (df['Decil_Risco'] > 1)]
    agregado_2 = filtrado_2.groupby('Codigo').agg({
        'Risco': 'mean',
        'Retorno_Anual': 'mean',
        'Decil_Retorno': 'mean',
        'Decil_Risco': 'mean',
        'Ano': 'mean'
    }).reset_index()
    lista_carteira_2.append(agregado_2)

    # Repetir para Carteiras 3, 4 e 5 com as condições apropriadas
    # Para a Carteira 3
    filtrado_3 = df[(df['Decil_Risco'] <= 3) & (df['Decil_Risco'] > 2)]
    agregado_3 = filtrado_3.groupby('Codigo').agg({
        'Risco': 'mean',
        'Retorno_Anual': 'mean',
        'Decil_Retorno': 'mean',
        'Decil_Risco': 'mean',
        'Ano': 'mean'
    }).reset_index()
    lista_carteira_3.append(agregado_3)

    # Para a Carteira 4
    filtrado_4 = df[(df['Decil_Risco'] <= 4) & (df['Decil_Risco'] > 3)]
    agregado_4 = filtrado_4.groupby('Codigo').agg({
        'Risco': 'mean',
        'Retorno_Anual': 'mean',
        'Decil_Retorno': 'mean',
        'Decil_Risco': 'mean',
        'Ano': 'mean'
    }).reset_index()
    lista_carteira_4.append(agregado_4)

    # Para a Carteira 5
    filtrado_5 = df[(df['Decil_Risco'] <= 5) & (df['Decil_Risco'] > 4)]
    agregado_5 = filtrado_5.groupby('Codigo').agg({
        'Risco': 'mean',
        'Retorno_Anual': 'mean',
        'Decil_Retorno': 'mean',
        'Decil_Risco': 'mean',
        'Ano': 'mean'
    }).reset_index()
    lista_carteira_5.append(agregado_5)


In [77]:
lista_trade_neutro = []
pd.set_option('display.max_columns', 100)
for i in range(2):
    df_filtrado = lista_dfs[i+1][lista_dfs[i+1]['Codigo'].isin(Lista_Carteira_Mercado[i]['Codigo'])]
    df_agrupado = df_filtrado.groupby('Ano').agg(Retorno_Carteira=('Retorno_Anual', lambda x: x.mean(skipna=True))).reset_index()
    df_agrupado['Ret_Acumulado_Carteira'] = df_agrupado['Retorno_Carteira'].cumprod()
    lista_trade_neutro.append(df_agrupado)

In [82]:
df_filtrado[df_filtrado['Ano'] == 2001]
df_agrupado['Retorno_Anual']

KeyError: 'Retorno_Anual'

In [79]:
df_filtrado

,Codigo,Ano,Datas,Total_Dias_Negociacao,Pregoes,Pregoes_Percentual,Retorno,Risco,Retorno_positivos,Retorno_negativos,Perc_positivos,Perc_negativos,Retorno_Anual,Retorno_acumulado,FIP,Decil_Retorno,Decil_Risco,Decil_FIP,Media_Negocios_Acao,Dias_Negocios,Negocios_Percentual,Media_Negocios,Razao_Media_Negocios,Media_Financeiro_Ativo_Ano,Financeiro_Dias,Total_Financeiro_Dias,Financeiro_Dias_Percentual,Media_Financeiro_Ano,DP_Financeiro_Ano,Razao_Financeiro
0,ACES4,2002,2002-01-01,261,249,0.954023,0.0,0.444863,104,101,0.507317,0.492683,0.184866,-0.434859,-0.014634,8,5,3,136.763052,249,0.954023,142.781241,0.957850,1125.754570,249,261,0.954023,4473.7592,11167.317339,0.251635
1,AMBV4,2002,2002-01-01,261,249,0.954023,0.0,0.331574,118,123,0.489627,0.510373,0.028484,0.618397,0.020747,7,2,4,176.044177,249,0.954023,142.781241,1.232964,10247.419157,249,261,0.954023,4473.7592,11167.317339,2.290561
2,ARCE4,2002,2002-01-01,261,249,0.954023,0.0,0.348041,122,102,0.544643,0.455357,1.241769,0.358895,-0.089286,10,3,1,13.738956,249,0.954023,142.781241,0.096224,529.171801,249,261,0.954023,4473.7592,11167.317339,0.118283
3,ARCZ6,2002,2002-01-01,261,249,0.954023,0.0,0.427253,131,109,0.545833,0.454167,0.592570,-0.218316,-0.091667,9,4,1,89.401606,249,0.954023,142.781241,0.626144,3243.414217,249,261,0.954023,4473.7592,11167.317339,0.724986
4,BBAS3,2002,2002-01-01,261,249,0.954023,0.0,0.544680,109,134,0.448560,0.551440,-0.123807,0.221589,0.102881,6,7,9,176.811245,249,0.954023,142.781241,1.238337,1645.675084,249,261,0.954023,4473.7592,11167.317339,0.367851
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,VALE5,2002,2002-01-01,261,249,0.954023,0.0,0.311459,139,100,0.581590,0.418410,0.664979,0.136653,-0.163180,9,1,1,215.726908,249,0.954023,142.781241,1.510891,20717.291570,249,261,0.954023,4473.7592,11167.317339,4.630846
74,VCPA4,2002,2002-01-01,261,249,0.954023,0.0,0.347559,112,122,0.478632,0.521368,0.356559,-0.103372,0.042735,9,3,5,78.092369,249,0.954023,142.781241,0.546937,2550.276077,249,261,0.954023,4473.7592,11167.317339,0.570052
75,VIVO4,2002,2002-01-01,261,249,0.954023,0.0,0.542670,112,133,0.457143,0.542857,-0.540555,-0.742737,0.085714,2,7,8,771.574297,249,0.954023,142.781241,5.403891,14255.123916,249,261,0.954023,4473.7592,11167.317339,3.186386
76,VIVT3,2002,2002-01-01,261,249,0.954023,0.0,0.335813,109,124,0.467811,0.532189,-0.050374,0.013862,0.064378,7,2,7,25.293173,249,0.954023,142.781241,0.177146,475.000112,249,261,0.954023,4473.7592,11167.317339,0.106175
